## Business Understanding

### Data Understanding

## Data Exploration

In [153]:
import numpy as np
import pandas as pd 
from numbers import Number
import sqlite3
from scipy import stats
import matplotlib.pyplot as plt

### Rotten Tomatoes (Marlo)

### The Numbers (Andy)

In [162]:
#read in the file
tn_raw_df = pd.read_csv("data/tn.movie_budgets.csv.gz")

In [163]:
tn_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [164]:
tn_raw_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


Clean data so that release_date and movie are strings, and production_budget, domestic_gross, worldwide_gross are integers

In [165]:
#run this cell only once
#production budget to int
tn_raw_df['production_budget'] = tn_raw_df["production_budget"].str.replace("$",'').str.replace(",",'')
tn_raw_df['production_budget'] = tn_raw_df["production_budget"].astype(int)

#domestic gross to int
tn_raw_df['domestic_gross'] = tn_raw_df["domestic_gross"].str.replace("$",'').str.replace(",",'')
tn_raw_df['domestic_gross'] = tn_raw_df["domestic_gross"].astype(int)

#worldwide_gross to int
tn_raw_df['worldwide_gross'] = tn_raw_df["worldwide_gross"].str.replace("$",'').str.replace(",",'')
tn_raw_df['worldwide_gross'] = tn_raw_df["worldwide_gross"].astype('int64')

tn_df = tn_raw_df.copy()

In [166]:
tn_df.describe()
tn_df.median()

id                         50.0
production_budget    17000000.0
domestic_gross       17225945.0
worldwide_gross      27984448.5
dtype: float64

In [167]:
tn_df.sort_values(by="domestic_gross", ascending=False)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
5,6,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,306000000,936662225,2053311220
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
41,42,"Feb 16, 2018",Black Panther,200000000,700059566,1348258224
6,7,"Apr 27, 2018",Avengers: Infinity War,300000000,678815482,2048134200
42,43,"Dec 19, 1997",Titanic,200000000,659363944,2208208395
...,...,...,...,...,...,...
2709,10,"Mar 31, 2004",The Touch,20000000,0,5918742
2708,9,"Apr 13, 2010",Three Kingdoms: Resurrection of the Dragon,20000000,0,22139590
2707,8,"Dec 31, 2012",Zambezia,20000000,0,34454336
2706,7,"Dec 31, 2008",Admiral,20000000,0,38585047


box office mojo domestic gross for star wars ep 7: 936700000.0
the numbers domestic gross for star wars ep 7: 936662225

In [192]:
tn_df.loc[tn_df['movie'] == "Titanic"]
bom_raw_df.loc[bom_raw_df['title'] == "Jaws"]

,title,studio,domestic_gross,foreign_gross,year


tn_db domestic gross: 198 680 470
tn_db worldwide gross: 390 525 192

bom lifetime gross: 200,882,193	
bom adjusted for inflation gross: 1,850,581,586	

In [168]:
tn_df.sort_values(by="production_budget", ascending=False)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,7000,0,0
5778,79,"Apr 2, 1999",Following,6000,48482,240495
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338
5780,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,0


### Box Office Mojo (Andy)

In [169]:
#read in data from box office mojo
bom_raw_df = pd.read_csv('data/bom.movie_gross.csv.gz')

In [170]:
bom_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [189]:
bom_raw_df.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [194]:
bom_raw_df.sort_values(by="domestic_gross", ascending=False)

,title,studio,domestic_gross,foreign_gross,year
1872,Star Wars: The Force Awakens,BV,936700000.0,"1,131.6",2015
3080,Black Panther,BV,700100000.0,646900000,2018
3079,Avengers: Infinity War,BV,678800000.0,"1,369.5",2018
1873,Jurassic World,Uni.,652300000.0,"1,019.4",2015
727,Marvel's The Avengers,BV,623400000.0,895500000,2012
...,...,...,...,...,...
1975,Surprise - Journey To The West,AR,NaN,49600000,2015
2392,Finding Mr. Right 2,CL,NaN,114700000,2016
2468,Solace,LGP,NaN,22400000,2016
2595,Viral,W/Dim.,NaN,552000,2016


In [173]:
bom_raw_df.describe()

,domestic_gross,year
count,3.359000e+03,3387.000000
mean,2.874585e+07,2013.958075
std,6.698250e+07,2.478141
min,1.000000e+02,2010.000000
25%,1.200000e+05,2012.000000
50%,1.400000e+06,2014.000000
75%,2.790000e+07,2016.000000
max,9.367000e+08,2018.000000


In [ ]:
bom_raw_df['foreign_gross'].isna().value_counts()

In [ ]:
bom_raw_df['studio'].value_counts()

### TheMovieDB (John)

### IMDB (John)

## Visualizations